In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# 🏥 Hospital Readmission Risk Analysis  
### Healthcare Analytics using Python

📌 **Version:** v1.0 – Exploratory Readmission Risk Analysis  
📅 **Last Updated:** February 2026  
🛠️ **Tools:** Python, Pandas, NumPy, Matplotlib  
📊 **Dataset:** Diabetes 130-US Hospitals (1999–2008)


## Business Understanding

Hospital readmissions increase healthcare costs and often indicate gaps in treatment, discharge planning, or patient follow-up care.

Reducing avoidable readmissions is a major priority for hospitals because it improves patient outcomes and reduces operational burden.

This project analyzes hospital encounter data to identify patterns associated with 30-day readmissions and support data-driven strategies for patient care improvement.


## 🎯 Objective

The objective of this project is to analyze hospital readmission patterns using
electronic health record (EHR) data and identify patient characteristics associated
with **30-day readmission risk**.

The goal is to generate **actionable insights** that can help healthcare providers
improve discharge planning and reduce avoidable readmissions.



### Key Performance Indicators (KPIs)

To evaluate hospital readmission patterns, the following KPIs are analyzed:

• Total patient encounters
• 30-day readmission rate
• Average length of hospital stay
• Average number of prior inpatient visits
• Medication usage intensity (number of medications)

These KPIs help identify high-risk patient groups and hospital utilization patterns linked to readmission risk.

## 📂 Dataset Overview

This dataset contains anonymized patient encounter records from U.S. hospitals
between **1999 and 2008**, focusing on diabetes-related admissions.

Each row represents a single hospital encounter and includes:
- Patient demographics
- Hospital utilization history
- Diagnostic information
- Medication usage
- Readmission outcome


## Dataset Loading

The dataset contains anonymized patient encounter records from U.S. hospitals (1999–2008) related to diabetes admissions.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('/kaggle/input/diabetes/diabetic_data.csv')
df.head()

In [ ]:
df.shape




In [ ]:
df.describe()

In [ ]:
df.info()

## Data Cleaning and Preparation

Data quality is critical in healthcare analytics to ensure reliable insights and decision-making.

The dataset contains missing values represented using special symbols (?). These were converted into proper missing values for accurate handling.

Columns with more than 50% missing data were removed, as they provide limited analytical value and may introduce bias.

Remaining missing values were retained or handled depending on analytical relevance.

Cleaning steps performed:

• Converted placeholder symbols to missing values
• Removed columns with excessive missing data
• Verified data types and structure
• Ensured consistency before analysis

## Missing Value Handling

Missing values in the dataset are represented using special symbols.
These are converted into proper missing values, and columns with excessive missing data are removed.


In [ ]:
df.replace('?', np.nan, inplace=True)

missing_percent = df.isna().mean() * 100
cols_to_drop = missing_percent[missing_percent > 50].index

df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

df.isna().mean().sort_values(ascending=False).head(10)


## 🎯 Target Variable Creation

The original `readmitted` column contains three categories:
- `<30` : Readmitted within 30 days
- `>30` : Readmitted after 30 days
- `NO`  : No readmission

For analysis, the target variable was converted into a **binary outcome**:
- **1** → Readmission within 30 days
- **0** → No readmission within 30 days

This allows clear comparison of high-risk vs low-risk patient groups.


In [ ]:
df['readmitted'].value_counts()

In [ ]:
df['readmitted_binary'] = df['readmitted'].apply(
    lambda x: 1 if x == '<30' else 0
)


In [ ]:
df['readmitted_binary'].value_counts()

In [ ]:
df.drop(columns=['readmitted'], inplace=True)


# 🧩 Feature Engineering

Feature engineering was performed to create meaningful analytical variables that better capture patient risk patterns.

Key transformations include:

• Conversion of the readmission outcome into a binary variable to identify high-risk patients (30-day readmission).
• Grouping of inpatient visit counts into categories to better observe utilization trends.
• Selection of clinically relevant variables to maintain analytical clarity and reduce noise.

These steps improve interpretability and allow clearer comparison of patient groups during analysis.

In [ ]:
df.columns

In [ ]:
selected_cols = [
    'age',
    'gender',
    'race',
    'time_in_hospital',
    'num_lab_procedures',
    'num_procedures',
    'num_medications',
    'number_outpatient',
    'number_emergency',
    'number_inpatient',
    'diag_1',
    'diag_2',
    'diag_3',
    'readmitted_binary'
]

df = df[selected_cols]


In [ ]:
df.shape
df.info()

In [ ]:
df['readmitted_binary'].mean()

In [ ]:
age_readmission = df.groupby('age')['readmitted_binary'].mean().sort_index()
age_readmission

## Exploratory Data Analysis

Exploratory analysis is performed to identify patterns and trends associated with hospital readmission risk.  
Visualizations are used to compare readmission rates across different patient groups.


### Readmission Rate by Age Group


In [ ]:
import matplotlib.pyplot as plt

age_readmission.plot(kind='bar', figsize=(10,5),color = 'orange')
plt.title('Readmission Rate by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Readmission Rate')
plt.show()


📈 Insight — Age Impact on Readmission

Readmission rates increase with age, especially among patients above 70 years. Older patients may require closer post-discharge monitoring and follow-up care to reduce readmission risk.

In [ ]:
df.groupby('number_inpatient')['readmitted_binary'].mean().head(10)


In [ ]:
df.loc[:,'inpatient_bin'] = pd.cut(
    df['number_inpatient'],
    bins=[-1, 0, 2, 5, 100],
    labels=['0 visits', '1–2 visits', '3–5 visits', '6+ visits']
)


In [ ]:
df[['number_inpatient', 'inpatient_bin']].head()


### Readmission Rate by Prior Inpatient Visits


In [ ]:
inpatient_readmission = (
    df.groupby('inpatient_bin', observed=True)['readmitted_binary']
      .mean()
)

inpatient_readmission.plot(kind='bar', figsize=(8,5))
plt.title('Readmission Rate by Prior Inpatient Visits')
plt.xlabel('Prior Inpatient Visits')
plt.ylabel('Readmission Rate')
plt.show()


📈 Insight — Prior Hospitalization Effect

Patients with frequent prior inpatient visits show significantly higher readmission risk, indicating chronic or severe health conditions requiring improved care planning.

In [ ]:
gender_readmission = (
    df.groupby('gender', observed=True)['readmitted_binary']
      .mean()
)
gender_readmission


### Readmission Rate by Gender


In [ ]:
gender_readmission.plot(kind='bar', figsize=(6,5),color = 'green')
plt.title('Readmission Rate by Gender')
plt.xlabel('Gender')
plt.ylabel('Readmission Rate')
plt.show()


📈 Insight — Gender Comparison

Readmission rates are similar across genders, suggesting gender alone is not a strong predictor of readmission risk.

## 🧠 Key Insights

- Readmission risk increases with patient age, particularly among older age groups
- Patients with frequent prior inpatient visits show significantly higher
  readmission rates
- Gender alone does not strongly influence 30-day readmission outcomes

## 🏥 Business Recommendations

Based on the analysis, the following strategies may help reduce hospital readmissions:

- Provide enhanced discharge planning and follow-up care for elderly patients.
- Closely monitor patients with prior inpatient admissions, as they show higher readmission risk.
- Improve patient education regarding medication adherence and post-discharge care.
- Strengthen outpatient and preventive care programs to reduce repeat hospital visits.

These measures can support hospitals in improving patient outcomes while reducing avoidable readmissions.

## ✅ Conclusion

This analysis highlights the importance of **age and hospital utilization history**
in predicting readmission risk.

The findings demonstrate how exploratory healthcare analytics can support
**data-driven decision-making**, helping providers identify high-risk patients
and improve discharge planning strategies.
